# dp100_11 Azure MLを使用して機械学習モデルをトレーニングする

## トレーニングスクリプトを実行する

- スクリプトにはトレーニング済みモデルを保存するコードを入れておき、`run.log`で精度計算結果をロギング
    - `joblib.dump(value=model, filename='outputs/model.pkl')`
- `ScriptRunConfig`で実行するスクリプト、環境、ディレクトリを指定する
- ハイパーパラメータ調整したい場合、`argparse`を用いて引数で渡すことも可能
    - `ScriptRunConfig`に引数`arguments = ['--reg-rate', 0.1]`

### スクリプト

```
from azureml.core import Run
import argparse
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# 実験の実行コンテキストの取得
run = Run.get_context()

# 正規化のハイパーパラメータを取得
parser = argparse.ArgumentParser()
parser.add_argument('--reg-rate', type=float, dest='reg_rate', default=0.01)
args = parser.parse_args()
reg = args.reg_rate

# データセットの準備
diabetes = pd.read_csv('data.csv')
X, y = data[['Feature1','Feature2','Feature3']].values, data['Label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# ロジスティック回帰モデルのトレーニング
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度計算
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# トレーニング済みモデルの保存
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/model.pkl')

run.complete()
```

### ScriptRunConfig

```
# スクリプトコンフィグの作成
script_config = ScriptRunConfig(source_directory='training_folder',
                                script='training.py',
                                arguments = ['--reg-rate', 0.1],
                                environment=sklearn_env)
```

## モデルの登録

- モデルをpikleでダンプしておけば、`download_files`メソッドでローカルシステムにダウンロードできる
- ダンプしたpikleファイルをモデルとして登録することができる。
    - プロパティに精度など入れておくと管理しやすい
    - ローカルファイルからモデルを登録するなら、`Model.register()`
    - モデル実行した**Run**への参照先があるなら、`Run.register_model()`
- `Model.list(Workspace)`で登録済みモデルを表示できる

### モデルファイルを取得する

```
# 'run'は完了した実験の実行を意味する

# 実験で生成されたファイルをリストアップ
for file in run.get_file_names():
    print(file)

# 名前付きファイルのダウンロード
run.download_file(name='outputs/model.pkl', output_file_path='model.pkl')
```

モデルの登録により、モデルの複数のバージョンを追跡し、推論(新しいデータからラベル地を予測)のモデルを取得できるようになる。  
モデルを登録するときに、名前、説明、タグ、フレームワーク(scikit-learnやpytorchなど)、フレームワークのバージョン、カスタムプロパティ、  
その他の便利なメタデータを指定できる。  
既存のモデルと同じ名前のモデルを登録すると、1から始まり、1ずつ増加する、そのモデルの新しいバージョンが自動的に作成される。

ローカルファイルからモデルを登録する場合は、ここに示すように、**Model**オブジェクトの**register**メソッドを使用できる。

```
from azureml.core import Model

model = Model.register(workspace=ws,
                       model_name='classification_model',
                       model_path='model.pkl', # local path
                       description='A classification model',
                       tags={'data-format': 'CSV'},
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.20.3')
```

また、モデルのトレーニングに使用された**Run**への参照がある場合は、ここに示すように**register_model**メソッドを使用できる。

```
run.register_model( model_name='classification_model',
                    model_path='outputs/model.pkl', # run outputs path
                    description='A classification model',
                    tags={'data-format': 'CSV'},
                    model_framework=Model.Framework.SCIKITLEARN,
                    model_framework_version='0.20.3')
                    
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Parameterized script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})
```

### 登録済みモデルを表示する

AzureMLスタジオでは、登録済みモデルを表示できる。  
**Model**オブジェクトを使用して、次のような登録済みモデルの詳細を取得することもできる。

```
from azureml.core import Model

for model in Model.list(ws):
    # Get model name and auto-generated version
    print(model.name, 'version:', model.version)
```